# Project 1: Gradient-based Algorithms and Differentiable Programming
## By: Daniel Rivera | MAE 598: Design Optimization
## Prof. Max Yi Ren | Fall 2022
#### This optimization problem involves the following:

$$ d(t+1) = d(t) +v(t)\Delta t$$
$$ v(t+1) =v(t) + a(t)\Delta t$$ 
#### and
$$ a(t) = f_{theta}(x(t)) $$

#### where d is the distance from the ground at any given time, v is the velocity at any given time, a is the acceleration at any given time and f is a neural network that has $$ \theta $$ as a parameter. Our goal is to optimize this formulation of a rocket landing through the utilization of all of the aformentioned formulas. The optimization problem is as follows: 

$$ min_{\theta} \begin{Vmatrix} x(T) \end{Vmatrix}^2 $$
$$ d(t+1) = d(t) +v(t)\Delta t$$
$$ v(t+1) =v(t) + a(t)\Delta t$$ 
$$ a(t) = f_{\theta}(x(t)), \forall t=1,...,T-1 $$

### Implementing Drag into the code
#### We can make this rocket-landing problem more realistic by adding a variable called drag, which is an opposing force that an aircraft's wings experience in flight and especially when landing. This force is defined mathematically as:

$$ F_{D}=\frac{C_{D}\rho V^{2}A}{2} $$

#### Where we then simplify the Variable Drag even further as 

$$ Drag= \frac{C_{D}V^{2}}{2} $$

#### Then our new optimization problem becomes:

$$ min_{\theta}(Drag,\Delta Drag, x(\theta))\begin{Vmatrix}x(T)\end{Vmatrix}^2 $$
$$ d(t+1) = d(t) +v(t)+Drag\Delta t$$
$$ v(t+1) =v(t) + a(t)+Drag\Delta t$$ 
$$ a(t) = f_{\theta}(x(t)), \forall t=1,...,T-1 $$
#### Let us just take a sample of 30 points of state:
$$ min_(\Theta)1/30\sum\limits_{i=1}^{30}(x^(i)(T))^2



#### Thus we've established an uncontrained problem with respect to theta.With that being said, we are able to make slight adjustments to the provided python script in order to determine solutions to this optimization problem. Several python libraries including pytortch and matplot lib are heavily utilized in order to simplify the coding. Furthermore, the matplot library is used in order to provide graphical representation of the solutions.Comments of each step within the code are provided along with a better problem formulation, and an analysis of findings at the end. 


In [16]:
#Import all of the necessary libraries 
# overhead

import logging
import math
import random
import numpy as np
import time
import torch as t
import torch.nn as nn
from torch import optim
from torch.nn import utils
import matplotlib.pyplot as plt

logger = logging.getLogger(__name__)

In [17]:
# environment parameters including the time interval, 
#and various accelerations

FRAME_TIME = 0.1  # time interval
GRAVITY_ACCEL = 0.12  # gravity constant
BOOST_ACCEL = 0.18  # thrust constant
DRAG_ACCEL = .06
# # the following parameters are not being used in the sample code
# PLATFORM_WIDTH = 0.25  # landing platform width
# PLATFORM_HEIGHT = 0.06  # landing platform height
# ROTATION_ACCEL = 20  # rotation constant

In [18]:
# define system dynamics
# Notes: 
# 0. You only need to modify the "forward" function
# 1. All variables in "forward" need to be PyTorch tensors.
# 2. All math operations in "forward" has to be differentiable, e.g., default PyTorch functions.
# 3. Do not use inplace operations, e.g., x += 1. Please see the following section for an example that does not work.

class Dynamics(nn.Module):

    def __init__(self):
        super(Dynamics, self).__init__()

    @staticmethod
    def forward(state, action):

        """
        action: thrust or no thrust
        state[0] = y
        state[1] = y_dot
        """
        
        # Apply gravity
        # Note: Here gravity is used to change velocity which is the second element of the state vector
        # Normally, we would do x[1] = x[1] + gravity * delta_time
        # but this is not allowed in PyTorch since it overwrites one variable (x[1]) that is part of the computational graph to be differentiated.
        # Therefore, I define a tensor dx = [0., gravity * delta_time], and do x = x + dx. This is allowed... 
        delta_state_gravity = t.tensor([0., GRAVITY_ACCEL * FRAME_TIME])

        # Thrust
        # Note: Same reason as above. Need a 2-by-1 tensor.
        delta_stateT = BOOST_ACCEL * FRAME_TIME * t.tensor([0., -1.]) * action
        
        # Drag
        delta_stateD = state * DRAG_ACCEL * FRAME_TIME * t.tensor([0., 1.]) * action
        
        delta_state = delta_stateT + delta_stateD
        
        # Update velocity
        state = state + delta_state + delta_state_gravity 
        
        # Update state
        # Note: Same as above. Use operators on matrices/tensors as much as possible. Do not use element-wise operators as they are considered inplace.
        step_mat = t.tensor([[1., FRAME_TIME],
                            [0., 1.]])
        state = t.matmul(step_mat, state)

        return state

In [19]:
# Adding third dimension work and progress
class Dynamics(nn.Module):

    def __init__(self):
        super(Dynamics, self).__init__()

    @staticmethod
    def forward(state, action):

       
        delta_state_gravity = t.tensor([0., GRAVITY_ACCEL * FRAME_TIME, 0.])

        # Thrust
        # Note: Same reason as above. Need a 2-by-1 tensor.
        delta_stateT = BOOST_ACCEL * FRAME_TIME * t.tensor([0., -1., 0.]) * action
        
        # Drag
        delta_stateD = state * DRAG_ACCEL * FRAME_TIME * t.tensor([0., 0., 1]) * action
        
        #delta_state = delta_stateT - delta_stateD
        
        # Update velocity
        state = state + delta_stateT + delta_state_gravity + delta_stateD
        
        # Update state
        # Note: Same as above. Use operators on matrices/tensors as much as possible. Do not use element-wise operators as they are considered inplace.
        step_mat = t.tensor([[1., FRAME_TIME, 1.],
                            [0., 1., 0.]])
        state = t.matmul(step_mat, state)

        return state

In [20]:
# a deterministic controller
# Note:
# 0. You only need to change the network architecture in "__init__"
# 1. nn.Sigmoid outputs values from 0 to 1, nn.Tanh from -1 to 1
# 2. You have all the freedom to make the network wider (by increasing "dim_hidden") or deeper (by adding more lines to nn.Sequential)
# 3. Always start with something simple
#t.transpose(xthrust_t.unsqueeze(0),0,1)

class Controller(nn.Module):

    def __init__(self, dim_input, dim_hidden, dim_output):
        """
        dim_input: # of system states
        dim_output: # of actions
        dim_hidden: up to you
        """
        super(Controller, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(dim_input, dim_hidden),
            nn.Tanh(),
            nn.Linear(dim_hidden, dim_output),
            # You can add more layers here
            nn.Sigmoid()
        )
#nn.Flatten()
    def forward(self, state):
        action = self.network(state)
        return action

In [21]:
# the simulator that rolls out x(1), x(2), ..., x(T)
# Note:
# 0. Need to change "initialize_state" to optimize the controller over a distribution of initial states
# 1. self.action_trajectory and self.state_trajectory stores the action and state trajectories along time

class Simulation(nn.Module):

    def __init__(self, controller, dynamics, T):
        super(Simulation, self).__init__()
        self.state = self.initialize_state()
        self.controller = controller
        self.dynamics = dynamics
        self.T = T 
        self.action_trajectory = []
        self.state_trajectory = []

    def forward(self, state):
        self.action_trajectory = []
        self.state_trajectory = []
        for _ in range(T):
            action = self.controller.forward(state)
            state = self.dynamics.forward(state, action)
            self.action_trajectory.append(action)
            self.state_trajectory.append(state)
        return self.error(state)

    @staticmethod
    def initialize_state():
        state = [1., 0.] 
        return t.tensor(state, requires_grad=False).float()

# For the second column of the state function there were several values that were tested. As the number increased into the real 
# whole number the loss increases exponentially.
# Values tested 0, 1, 2, 3, 4, 5, 6,

    def error(self, state):
        return state[0]**2 + state[1]**2 

In [22]:
# set up the optimizer
# Note:
# 0. LBFGS is a good choice if you don't have a large batch size (i.e., a lot of initial states to consider simultaneously)
# 1. You can also try SGD and other momentum-based methods implemented in PyTorch
# 2. You will need to customize "visualize"
# 3. loss.backward is where the gradient is calculated (d_loss/d_variables)
# 4. self.optimizer.step(closure) is where gradient descent is done

class Optimize:
    def __init__(self, simulation):
        self.simulation = simulation
        self.parameters = simulation.controller.parameters()
        self.optimizer = optim.LBFGS(self.parameters, lr=0.01)

    def step(self):
        def closure():
            loss = self.simulation(self.simulation.state)
            self.optimizer.zero_grad()
            loss.backward()
            return loss
        self.optimizer.step(closure)
        return closure()
    
    def train(self, epochs):
        L = []
        for epoch in range(epochs):
            loss = self.step()
            L.append(loss)
            print('[%d] loss: %.3f' % (epoch + 1, loss))
            self.visualize()
        return L

    def visualize(self):
    
    # Loss vs iteration
        data = np.array([self.simulation.state_trajectory[i].detach().numpy() for i in range(self.simulation.T)])
        x = data[:, 0]
        y = data[:, 1]
        plt.plot(x, y)
        plt.show()
        plt.title('Loss vs Iteration')
        plt.xlabel('Iteration')
        plt.ylabel('Loss')

In [23]:
# Now it's time to run the code!

T = 100  # number of time steps
dim_input = 2  # state space dimensions
dim_hidden = 8  # latent dimensions
dim_output = 2  # action space dimensions
d = Dynamics()  # define dynamics
c = Controller(dim_input, dim_hidden, dim_output)  # define controller
s = Simulation(c, d, T)  # define simulation
o = Optimize(s)  # define optimizer
o.train(40)  # solve the optimization problem

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 0